In [ ]:
#!pip install torch onnx tensorflow numpy matplotlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x_values = np.linspace(0, 2 * np.pi, 1000).reshape(-1, 1).astype(np.float32)
y_values = np.sin(x_values).astype(np.float32)

print(f"Data generated: {len(x_values)} points")

Export to .onnx

In [ ]:
import torch
import torch.nn as nn

try:
    class SineModelPyTorch(nn.Module):
        def __init__(self):
            super(SineModelPyTorch, self).__init__()
            self.net = nn.Sequential(
                nn.Linear(1, 16),
                nn.ReLU(),
                nn.Linear(16, 1)
            )
        def forward(self, x):
            return self.net(x)

    pt_model = SineModelPyTorch()
    pt_model.eval() 

    dummy_input = torch.tensor([[0.0]], dtype=torch.float32)

    torch.onnx.export(
        pt_model, 
        dummy_input, 
        "model.onnx", 
        export_params=True,       
        opset_version=12,          
        do_constant_folding=True,  
        input_names=['input'],     
        output_names=['output'],   
        dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}} 
    )
    print("Successfully exported to model.onnx")
except Exception as e:
    print(f"Error exporting ONNX: {e}")

Export to .tflite


In [ ]:
import tensorflow as tf

try:
    tf_model = tf.keras.Sequential([
        tf.keras.layers.Dense(16, activation='relu', input_shape=(1,)),
        tf.keras.layers.Dense(1)
    ])
    
    tf_model.build(input_shape=(None, 1))

    converter = tf.lite.TFLiteConverter.from_keras_model(tf_model)
    
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    
    tflite_model = converter.convert()

    with open("model.tflite", "wb") as f:
        f.write(tflite_model)

    print("Successfully exported to model.tflite")
except Exception as e:
    print(f"Error exporting TFLite: {e}")

Export to .h

In [ ]:
def hex_to_c_array(hex_data, var_name):
    c_str = f"unsigned char {var_name}[] = {{\n  "
    for i, byte in enumerate(hex_data):
        c_str += f"0x{byte:02x}, "
        if (i + 1) % 12 == 0:
            c_str += "\n  "
    c_str = c_str.strip().strip(",") + "\n};\n"
    c_str += f"unsigned int {var_name}_len = {len(hex_data)};"
    return c_str

if 'tflite_model' in locals():
    with open("model.h", "w") as f:
        f.write(hex_to_c_array(tflite_model, "sine_model_data"))
    print("Successfully exported to model.h")
else:
    print("TFLite model not found, skipping .h export")

Export to .bin

In [ ]:
try:
    weights_list = []
    for param in pt_model.parameters():
        weights_list.append(param.detach().cpu().numpy().flatten())

    all_weights = np.concatenate(weights_list)
    
    with open("model.bin", "wb") as f:
        f.write(all_weights.tobytes())

    print(f"Successfully exported weights to model.bin ({all_weights.nbytes} bytes)")
except Exception as e:
    print(f"Error exporting .bin: {e}")